# Training the competition model

Firstly load all data.

In [2]:
original_json_file_path = "./datasets/competition_final/generated/final_year_1_to_10_data.json"
year_11_json_file_path = "./datasets/competition_final/generated/final_year_11_data.json"

In [3]:
import pandas as pd

original_json_file_path = "./datasets/competition_final/generated/final_year_1_to_10_data.json"
year_11_json_file_path = "./datasets/competition_final/generated/final_year_11_data.json"

df_original = pd.read_json(original_json_file_path)
df_year_11 = pd.read_json(year_11_json_file_path)

df_year_11.head(20)

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,11,NaN,0.470588,244.629371,122.927739,47.384615,29.587413,12.310023,50.428904,0.418658,0.749509,0.280573,0.500000,2
1,TUL,11,NaN,0.500000,196.262500,87.358333,39.875000,19.675000,8.566667,38.229167,0.414552,0.780216,0.327587,0.500000,10
2,SEA,11,NaN,0.588235,274.577622,106.679720,55.923077,26.044755,11.078322,49.041958,0.412312,0.802031,0.310246,0.588235,8
3,SAS,11,NaN,0.441176,318.134545,111.044848,67.100000,34.910909,14.202424,50.506061,0.428546,0.808280,0.340362,0.441176,2
4,PHO,11,NaN,0.676471,259.051748,113.709557,49.538462,21.234965,13.693240,47.233100,0.444417,0.801936,0.335072,0.676471,6
5,NYL,11,NaN,0.382353,239.084416,93.432900,46.714286,19.831169,8.073593,40.969697,0.396254,0.808497,0.311745,0.428571,2
6,MIN,11,NaN,0.411765,285.395041,127.131680,61.545455,30.919008,10.638567,50.823691,0.446331,0.781802,0.320708,0.500000,3
7,LAS,11,NaN,0.529412,249.046281,122.257851,66.000000,26.052893,14.456198,54.231405,0.420500,0.807764,0.311575,0.500000,6
8,IND,11,NaN,0.647059,284.894215,111.692011,51.090909,33.229752,16.182369,50.046832,0.409415,0.817711,0.328738,0.647059,6
9,CON,11,NaN,0.470588,239.584848,93.972727,49.500000,22.596970,9.503030,45.674242,0.430382,0.791903,0.286320,0.470588,1


Train the model

## Logistic Regression

In [14]:
# Model training...
from itertools import combinations, product
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Assume df is your DataFrame
# Define features and target
features = ['averageWinRate', 'averagePoints', 'averageRebounds', 'averageAssists',
            'averageSteals', 'averageBlocks', 'averageTurnovers', 'averageFGRatio',
            'averageFTRatio', 'averageThreeRatio', 'coachWinRate', 'numberOfAwardedPlayers']
target = 'playoff'

# Split data into training and test sets
X_train = df_original[df_original["year"] < 7][features + ['year']]
X_test = df_original[df_original["year"] >= 7][features + ['year']]
y_train = df_original[df_original["year"] < 7]["playoff"]
y_test = df_original[df_original["year"] >= 7]["playoff"]

df_year_only_train = X_train["year"]
df_year_only_test = X_test["year"]

best_accuracy_sum = 0
best_feature_combination = None
best_hyperparameters = None

# Define hyperparameters grid
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
    'tol': [1e-4, 1e-3, 1e-2],
    'max_iter': [10000]
}

# Generate all combinations of hyperparameters
all_params = [dict(zip(param_grid, v)) for v in product(*param_grid.values())]

# Iterate through all possible feature combinations with 2 to 6 features
results = pd.DataFrame(columns=['acc_train', 'acc_test', 'best_feature_combination', 'best_hyperparameters'])

for subset in tqdm(combinations(features, 2), desc="Feature Combinations"):
    selected_features = list(subset)
    selected_features.append('year')  # Always include 'year

    X_train_subset = X_train[selected_features]
    X_test_subset = X_test[selected_features]

    for params in all_params:
        # Create a Logistic Regression model with given hyperparameters
        model = LogisticRegression(**params)
        model.fit(X_train_subset, y_train)
        
        X_train = pd.concat([X_train, df_year_only_train], axis=1)
        X_test = pd.concat([X_test, df_year_only_test], axis=1)

        # Evaluate the model on the test set
        y_pred_test = model.predict(X_test_subset)
        accuracy_test = accuracy_score(y_test, y_pred_test)
        y_pred_train = model.predict(X_train_subset)
        accuracy_train = accuracy_score(y_train, y_pred_train)

        # Check if this combination is the best so far
        accuracy_sum = accuracy_train + accuracy_test

        if accuracy_test > 0.65:
            temp_row = pd.DataFrame([{
                'acc_train': accuracy_train,
                'acc_test': accuracy_test,
                'best_feature_combination': str(selected_features),
                'best_hyperparameters': str(params)
            }])
            results = pd.concat([results, temp_row])
    
    break

df_cd_json = results.to_json(orient="records")
new_file = open("./datasets/competition_final/generated/model_evaluation_by_accuracy.json", "w")
new_file.writelines(df_cd_json)
new_file.close()

results.head()
 

Best Hyperparameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.6666666666666666


,acc_train,acc_test,best_feature_combination,best_hyperparameters


## SVM

In [15]:
# Model training...
from itertools import combinations, product
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.svm import SVC

# Assume df is your DataFrame
# Define features and target
features = ['averageWinRate', 'averagePoints', 'averageRebounds', 'averageAssists',
            'averageSteals', 'averageBlocks', 'averageTurnovers', 'averageFGRatio',
            'averageFTRatio', 'averageThreeRatio', 'coachWinRate', 'numberOfAwardedPlayers']
target = 'playoff'

# Split data into training and test sets
X_train = df_original[df_original["year"] < 7][features + ['year']]
X_test = df_original[df_original["year"] >= 7][features + ['year']]
y_train = df_original[df_original["year"] < 7]["playoff"]
y_test = df_original[df_original["year"] >= 7]["playoff"]

df_year_only_train = X_train["year"]
df_year_only_test = X_test["year"]

best_accuracy_sum = 0
best_feature_combination = None
best_hyperparameters = None

# Define hyperparameters grid
param_grid = {
    'C': [0.1, 1, 10, 100],        # Regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel type
    'gamma': ['scale', 'auto'],   # Kernel coefficient
}

# Generate all combinations of hyperparameters
all_params = [dict(zip(param_grid, v)) for v in product(*param_grid.values())]

# Iterate through all possible feature combinations with 2 to 6 features
results = pd.DataFrame(columns=['acc_train', 'acc_test', 'best_feature_combination', 'best_hyperparameters'])

# Always include 'year

X_train = pd.concat([X_train, df_year_only_train], axis=1)
X_test = pd.concat([X_test, df_year_only_test], axis=1)

svm = SVC(kernel='linear')  # You can change the kernel as needed (e.g., 'rbf' for radial basis function)

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')

# Fit the model to the training data
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Make predictions on the test set using the best model
y_pred = grid_search.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
    
results.head()
 

## KNN Evaluation idk

In [59]:
import pandas as pd

file = pd.read_json("./datasets/generated/model_evaluation_by_accuracy_knn.json")

file["diff"] = abs(file["acc_train"] - file["acc_test"])

df2 = file[(file["acc_test"] > 0.75) & (file["diff"] < 0.05)].sort_values("acc_test", axis=0, ascending=False)

df2.head()

print(df2.iloc[0,2])
print(df2.iloc[1,2])
print(df2.iloc[2,2])
print(df2.iloc[3,2])
print(df2.iloc[4,2])
print(df2.iloc[5,3])
print(df2.iloc[6,2])



['averageAssists', 'averageTurnovers', 'numberOfAwardedPlayers', 'year', 'year']
['averageAssists', 'averageTurnovers', 'numberOfAwardedPlayers', 'year', 'year']
['averageAssists', 'averageTurnovers', 'averageThreeRatio', 'coachWinRate', 'numberOfAwardedPlayers', 'year', 'year']
['averageAssists', 'averageTurnovers', 'averageFTRatio', 'coachWinRate', 'numberOfAwardedPlayers', 'year', 'year']
['averageAssists', 'averageTurnovers', 'averageFTRatio', 'coachWinRate', 'numberOfAwardedPlayers', 'year', 'year']
{'n_neighbors': 7, 'weights': 'uniform', 'metric': 'euclidean'}
['averageAssists', 'averageTurnovers', 'averageFTRatio', 'averageThreeRatio', 'numberOfAwardedPlayers', 'year', 'year']


## Logreg evaluation

In [55]:
#load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd

data_file_path = "./datasets/competition_final/generated/final_year_1_to_10_data.json"
df = pd.read_json(data_file_path)
df.head()

features = ['averageWinRate', 'averageRebounds', 'averageBlocks', 'averageThreeRatio', 'coachWinRate', 'numberOfAwardedPlayers', 'year']

results_df = pd.DataFrame(columns=["year_split", "acc_test", "acc_train", "diff"])

for i in range(5,11):
  #Split dataset in such way that we use latest years for training and older years for testing
  X_train=df[df["year"]<i][features]
  X_test=df[df["year"]>=i][features]
  y_train=df[df["year"]<i]["playoff"]
  y_test=df[df["year"]>=i]["playoff"]

  models = {}

  #Logistic Regression
  models['Logistic Regression'] = LogisticRegression(C=1, penalty= 'l2', solver='newton-cg', tol=0.001, max_iter=10000)

  accuracy, precision, recall = {}, {}, {}

  # Fit the classifier
  models['Logistic Regression'].fit(X_train, y_train)

  # Make predictions
  y_pred = models['Logistic Regression'].predict(X_test)

  y_pred_test = models['Logistic Regression'].predict(X_train)

  temp_df = pd.DataFrame([[i, metrics.accuracy_score(y_test, y_pred), metrics.accuracy_score(y_train, y_pred_test), abs(metrics.accuracy_score(y_test, y_pred) - metrics.accuracy_score(y_train, y_pred_test))]], columns=["year_split", "acc_test", "acc_train", "diff"])
  
  results_df = pd.concat([results_df, temp_df])

"""   # Calculate metrics
  print("Test Accuracy: ", metrics.accuracy_score(y_test, y_pred))
  print("Test Precision: ", metrics.precision_score(y_test, y_pred, pos_label='Y'))
  print("Test Recall: ", metrics.recall_score(y_test, y_pred, pos_label='Y'))

  print("Train Accuracy: ", metrics.accuracy_score(y_train, y_pred_test))
  print("Train Precision: ", metrics.precision_score(y_train, y_pred_test, pos_label='Y'))
  print("Train Recall: ", metrics.recall_score(y_train, y_pred_test, pos_label='Y')) """
  
results_df

results_df["acc_test"].mean()

0.6660322851554195

In [60]:
#load libraries
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd

data_file_path = "./datasets/competition_final/generated/final_year_1_to_10_data.json"
df = pd.read_json(data_file_path)
df.head()

features = ['averageAssists', 'averageTurnovers', 'averageFTRatio', 'averageThreeRatio', 'numberOfAwardedPlayers', 'year']

results_df = pd.DataFrame(columns=["year_split", "acc_test", "acc_train", "diff"])

for i in range(5,11):
  #Split dataset in such way that we use latest years for training and older years for testing
  X_train=df[df["year"]<i][features]
  X_test=df[df["year"]>=i][features]
  y_train=df[df["year"]<i]["playoff"]
  y_test=df[df["year"]>=i]["playoff"]


  models = {}

  #K-Nearest Neighbors
  models['K-Nearest Neighbor'] = KNeighborsClassifier(n_neighbors=7, weights='uniform', metric='euclidean')

  accuracy, precision, recall = {}, {}, {}

  # Fit the classifier
  models['K-Nearest Neighbor'].fit(X_train, y_train)

  # Make predictions
  y_pred = models['K-Nearest Neighbor'].predict(X_test)

  y_pred_test = models['K-Nearest Neighbor'].predict(X_train)

  temp_df = pd.DataFrame([[i, metrics.accuracy_score(y_test, y_pred), metrics.accuracy_score(y_train, y_pred_test), abs(metrics.accuracy_score(y_test, y_pred) - metrics.accuracy_score(y_train, y_pred_test))]], columns=["year_split", "acc_test", "acc_train", "diff"])
  
  results_df = pd.concat([results_df, temp_df])

  # Calculate metrics
  """ print("Test Accuracy: ", metrics.accuracy_score(y_test, y_pred))
  print("Test Precision: ", metrics.precision_score(y_test, y_pred, pos_label='Y'))
  print("Test Recall: ", metrics.recall_score(y_test, y_pred, pos_label='Y'))

  print("Train Accuracy: ", metrics.accuracy_score(y_train, y_pred_test))
  print("Train Precision: ", metrics.precision_score(y_train, y_pred_test, pos_label='Y'))
  print("Train Recall: ", metrics.recall_score(y_train, y_pred_test, pos_label='Y')) """
  
results_df

,year_split,acc_test,acc_train,diff
0,5,0.650000,0.782609,0.132609
0,6,0.686567,0.745763,0.059196
0,7,0.722222,0.791667,0.069444
0,8,0.575000,0.837209,0.262209
0,9,0.592593,0.818182,0.225589
0,10,0.692308,0.823009,0.130701


## Conclusion

Logreg is better because it is more consistent.

Best is year 7 split.

After year 8 for some reason in all cases the precision and/or difference in betweeen test and train accuracy.